In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn.metrics as metrics
from sklearn.naive_bayes import MultinomialNB

In [2]:
# calculate the entropy of a subset
def entropy(target_col):
    labels, counts = np.unique(target_col, return_counts=True)
    counts = list(counts)
    return np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))for i in range(len(labels))])

In [3]:
# calculate the information gain of a dataset
def InfoGain(data, split_feature, target_feature="Label"):
    pre_entropy = entropy(data[target_feature])
    
    # Get unique values in the split feature and their corresponding counts
    vals, counts = np.unique(data[split_feature], return_counts=True)
    counts = list(counts)
    post_entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_feature]==vals[i]).dropna()[target_feature]) for i in range(len(vals))])
    #print("Entropy: ",pre_entropy, post_entropy)
    return pre_entropy - post_entropy

In [4]:
def GainRatio(data, split_feature, target_feature="Label"):
    info_gain = InfoGain(data, split_feature, target_feature)
#     print("Info gain",info_gain)
    vals, counts = np.unique(data[split_feature], return_counts=True)
    counts = list(counts)
    #print(counts)
    #print("infos: ",[-(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(counts))])
    split_info = np.sum([-(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(counts))])
#     print(vals)
#     print("Split infos", [-(counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(counts))])
    return info_gain/split_info

In [5]:
def GiniIndex(target_col):
    labels, counts = np.unique(target_col, return_counts=True)
    counts = list(counts)
    return 1-np.sum([np.square((-counts[i]/np.sum(counts))) for i in range(len(labels))])

In [6]:
def Reduction_Gini(data, split_feature, target_feature="Label"):
    pre_gini = GiniIndex(data[target_feature])
    vals, counts = np.unique(data[split_feature], return_counts=True)
    counts = list(counts)
    post_gini = np.sum([counts[i]/np.sum(counts)*GiniIndex(data.where(data[split_feature]==vals[i]).dropna()[target_feature]) for i in range(len(counts))])
    #print(pre_gini, post_gini)
    return pre_gini-post_gini

In [7]:
# ID3 Algorithm: uses infomation as as splitting criteria
def ID3(data, original_data, cur_features,Label):
    tree = {}    
    info_gains = [InfoGain(data, sp) for sp in cur_features]
    #print(cur_features)
    #print(info_gains)
    if len(info_gains) > 0:
        best_feature_index = np.argmax(info_gains)
        best_feature = cur_features[best_feature_index]
        best_feature_vals = np.unique(data[best_feature])
    if data.empty:
        mode = np.unique(original_data[Label])[np.argmax(np.unique(original_data[Label],return_counts=True)[1])]
        return(mode)        
    elif len(np.unique(data[Label])) == 1:
        return np.unique(data[Label])[0]
    elif len(cur_features) == 0 or all(infogain==0 for infogain in list(info_gains)):
        vals,counts = np.unique(data[Label],return_counts=True)
        if all(count==counts[0] for count in counts):
            mode = np.unique(original_data[Label])[np.argmax(np.unique(original_data[Label],return_counts=True)[1])]
        else:
            mode = vals[np.argmax(counts)]
        return(mode)
    all_feature_vals = np.unique(original_data[best_feature])
    sub_datas = [data.where(data[best_feature]==val).dropna().drop(columns=best_feature) 
                 if val in list(data[best_feature]) else pd.DataFrame() for val in all_feature_vals]
    del cur_features[best_feature_index]
    for i,sub_data in enumerate(sub_datas):
        tree[all_feature_vals[i]] = ID3(sub_data, original_data, cur_features[:],Label)
    return tree

In [8]:
# C4.5 Algorithm: uses gain ratio as as splitting criteria
def C45(data, original_data, cur_features,Label):
    if data.empty:
        mode = np.unique(original_data[Label])[np.argmax(np.unique(original_data[Label],return_counts=True)[1])]
        return(mode)  
    
    tree = {}    
    gain_ratios = [GainRatio(data, sp) for sp in cur_features]
    #print("Current features:",cur_features)
    #print(gain_ratios)
    if len(gain_ratios) > 0:
        best_feature_index = np.argmax(gain_ratios)
        best_feature = cur_features[best_feature_index]
        best_feature_vals = np.unique(data[best_feature])      
    if len(np.unique(data[Label])) == 1:
        return np.unique(data[Label])[0]
    elif len(cur_features) == 0 or all(gainratio==0 for gainratio in list(gain_ratios)):
        vals,counts = np.unique(data[Label],return_counts=True)
        if all(count==counts[0] for count in counts):
            mode = np.unique(original_data[Label])[np.argmax(np.unique(original_data[Label],return_counts=True)[1])]
        else:
            mode = vals[np.argmax(counts)]
        return(mode)
    all_feature_vals = np.unique(original_data[best_feature])
    sub_datas = [data.where(data[best_feature]==val).dropna().drop(columns=best_feature) 
                 if val in list(data[best_feature]) else pd.DataFrame() for val in all_feature_vals]
    del cur_features[best_feature_index]
    for i,sub_data in enumerate(sub_datas):
        tree[all_feature_vals[i]] = C45(sub_data, original_data, cur_features[:],Label)
    return tree

In [9]:
# CART Algorithm: uses gini index as as splitting criteria
def CART(data, original_data, cur_features,Label):
    if data.empty:
        mode = np.unique(original_data[Label])[np.argmax(np.unique(original_data[Label],return_counts=True)[1])]
        return(mode)  
    tree = {}    
    ginis = [Reduction_Gini(data, sp) for sp in cur_features]
    #print("Current features:",cur_features)
    #print(ginis)
    if len(ginis) > 0:
        best_feature_index = np.argmax(ginis)
        best_feature = cur_features[best_feature_index]
        best_feature_vals = np.unique(data[best_feature])      
    if len(np.unique(data[Label])) == 1:
        return np.unique(data[Label])[0]
    elif len(cur_features) == 0 or all(gini==0 for gini in list(ginis)):
        vals,counts = np.unique(data[Label],return_counts=True)
        if all(count==counts[0] for count in counts):
            mode = np.unique(original_data[Label])[np.argmax(np.unique(original_data[Label],return_counts=True)[1])]
        else:
            mode = vals[np.argmax(counts)]
        return(mode)
    all_feature_vals = np.unique(original_data[best_feature])
    sub_datas = [data.where(data[best_feature]==val).dropna().drop(columns=best_feature) 
                 if val in list(data[best_feature]) else pd.DataFrame() for val in all_feature_vals]
    del cur_features[best_feature_index]
    for i,sub_data in enumerate(sub_datas):
        tree[all_feature_vals[i]] = CART(sub_data, original_data, cur_features[:],Label)
    return tree

In [10]:
def DecisionTreeClassifier(data, originalTree, Label=True):
    results = []
    if Label:
        features= list(train)[1:-1]
        Label = list(train)[-1]
    else:
        features= list(train)[1:-1]
    
    for i,row in data[features].iterrows():
        #print("Original:",originalTree)

        dtree = originalTree
        #print(row.keys())
        vals = list([row[key] for key in row.keys()])
        found = False
        while len(vals) > 0 and not found:
            #print(vals)
            #print("The tree right now:",dtree)
            for val in vals:
                if val in dtree.keys():
                    ind = vals.index(val)
                    #print("vals:",vals)
                    #print("val:",val)
                    #print("index:",ind)
                    vals.remove(val)
                    if not isinstance(dtree[val], dict):
                        #print(dtree[val])
                        results.append(dtree[val])
                        found = True
                    else:
                        dtree = dtree[val]
    return results

In [11]:
########### Question 1 ###############
train = pd.read_csv('q1_train.csv')
features= list(train)[1:-1]
Label = list(train)[-1]
tree_ID3 = ID3(train,train,features[:],Label)
tree_C45 = C45(train,train,features[:],Label)
tree_CART = CART(train,train,features[:],Label)
print("ID3 Tree:", tree_ID3)
print("C45 Tree:",tree_C45)
print("CART Tree:",tree_CART)
########### Question 1 ###############


ID3 Tree: {'In': 'Lose', 'Out': 'Win'}
C45 Tree: {'In': 'Lose', 'Out': 'Win'}
CART Tree: {'In': 'Lose', 'Out': 'Win'}


C:\Users\Yunch\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [12]:
########### Question 1 ###############
train = pd.read_csv('q2_train.csv')
features= list(train)[1:-1]
Label = list(train)[-1]
tree_ID3 = ID3(train,train,features[:],Label)
tree_C45 = C45(train,train,features[:],Label)
tree_CART = CART(train,train,features[:],Label)
print("ID3 Tree:", tree_ID3)
print("C45 Tree:",tree_C45)
print("CART Tree:",tree_CART)
########### Question 1 ###############

ID3 Tree: {'Overcast': 'Yes', 'Rainy': {False: 'Yes', True: 'No'}, 'Sunny': {'High': 'No', 'Normal': 'Yes'}}
C45 Tree: {'Overcast': 'Yes', 'Rainy': {False: 'Yes', True: 'No'}, 'Sunny': {'High': 'No', 'Normal': 'Yes'}}
CART Tree: {'Overcast': 'Yes', 'Rainy': {False: 'Yes', True: 'No'}, 'Sunny': {'High': 'No', 'Normal': 'Yes'}}


In [13]:
train = pd.read_csv('q5_train.csv')
test = pd.read_csv('q5_test.csv')
true_labels = list(test["Label"])

# Get feature names and label name
features= list(train)[1:-1]
Label = list(train)[-1]

tree_ID3 = ID3(train,train,features[:],Label)
tree_C45 = C45(train,train,features[:],Label)
tree_CART = CART(train,train,features[:],Label)
print(tree_ID3)
print(tree_C45)
print(tree_CART)

predicted_ID3 = DecisionTreeClassifier(test, tree_ID3)
predicted_C45 = DecisionTreeClassifier(test, tree_C45)
predicted_CART = DecisionTreeClassifier(test, tree_CART)

le = preprocessing.LabelEncoder()
le.fit(true_labels)
true_labels = le.transform(true_labels)
predicted_ID3 = le.transform(predicted_ID3)
predicted_C45 = le.transform(predicted_C45)
predicted_CART = le.transform(predicted_CART)


{'ABC': {'In': 'Lose', 'Out': 'Win'}, 'CBS': 'Lose', 'ESPN': 'Win', 'FOX': 'Lose', 'NBC': {'In': 'Win', 'Out': {'Away': 'Win', 'Home': 'Win'}}}
{'In': {'Away': 'Lose', 'Home': {'ABC': 'Win', 'CBS': 'Win', 'ESPN': 'Win', 'FOX': 'Win', 'NBC': 'Win'}}, 'Out': {'ABC': {'Away': 'Win', 'Home': 'Win'}, 'CBS': 'Lose', 'ESPN': 'Win', 'FOX': 'Win', 'NBC': {'Away': 'Win', 'Home': 'Win'}}}
{'ABC': {'In': 'Lose', 'Out': 'Win'}, 'CBS': 'Lose', 'ESPN': 'Win', 'FOX': 'Lose', 'NBC': {'In': 'Win', 'Out': {'Away': 'Win', 'Home': 'Win'}}}


C:\Users\Yunch\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [14]:
train_X = []
test_X = []
le = [preprocessing.LabelEncoder().fit(list(train[features[i]])) for i in range(len(features))]
for i, row in train[features].iterrows():
    train_X.append([le[j].transform([list(row)[j]])[0] for j in range(len(list(row)))])
for i, row in test[features].iterrows():
    test_X.append([le[j].transform([list(row)[j]])[0] for j in range(len(list(row)))])

y = list(train[Label])
le = preprocessing.LabelEncoder().fit(y)
y = le.transform(y)

clf = MultinomialNB()
clf.fit(np.array(train_X), y)
predicted_NB = clf.predict(test_X)

In [15]:
print("Test results with ID3 Classification Model:")
print("Accuracy: ", metrics.accuracy_score(true_labels, predicted_ID3))
print("Precision: ", metrics.precision_score(true_labels, predicted_ID3))
print("Recall: ", metrics.recall_score(true_labels, predicted_ID3))
print("F1: ", metrics.f1_score(true_labels, predicted_ID3))
print()
print("Test results with C45 Classification Model:")
print("Accuracy: ", metrics.accuracy_score(true_labels, predicted_C45))
print("Precision: ", metrics.precision_score(true_labels, predicted_C45))
print("Recall: ", metrics.recall_score(true_labels, predicted_C45))
print("F1: ", metrics.f1_score(true_labels, predicted_C45))
print()
print("Test results with Naive Bayes Classification Model:")
print("Accuracy: ", metrics.accuracy_score(true_labels, predicted_NB))
print("Precision: ", metrics.precision_score(true_labels, predicted_NB))
print("Recall: ", metrics.recall_score(true_labels, predicted_NB))
print("F1: ", metrics.f1_score(true_labels, predicted_NB))

Test results with ID3 Classification Model:
Accuracy:  0.8333333333333334
Precision:  0.8888888888888888
Recall:  0.8888888888888888
F1:  0.8888888888888888

Test results with C45 Classification Model:
Accuracy:  0.9166666666666666
Precision:  0.9
Recall:  1.0
F1:  0.9473684210526316

Test results with Naive Bayes Classification Model:
Accuracy:  0.75
Precision:  0.75
Recall:  1.0
F1:  0.8571428571428571
